In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import sympy as sym
from pylab import *
from scipy.ndimage import measurements
from copy import copy, deepcopy

In [ ]:
N = 30 #sets the grid size 
x = np.random.rand(N,N)

In [ ]:
for i in range(0,N):
    for j in range(0,N):
        if x[i,j] < 0.6:
            x[i,j]=1
        else:
            x[i,j]=-1 #Generates a T= inf state

In [ ]:
#x=np.ones((N,N)) # Generates a array with all +1; i.e , a T=0 state

In [ ]:
plt.imshow(x)
#plt.colorbar() #Visualising the state

<center> Thus we generate $x$ which is a NxN random array of 1 and -1  

In [ ]:
#a=np.zeros((50,50))
#a[a==0]=-1
      #for i in range(50):
    #for j in range(50):
       # if i==j or i==(50-j):
        #    a[i,j]=+1

In [ ]:
#plt.imshow(a)

In [ ]:
#avg_size(a)

In [ ]:
# Gives us the energy associated with single spin
def spin(i,j):
    if i==N-1 and j==N-1:
        return x[i,j]*(x[i,j-1] + x[0,j] + x[i,0] + x[i-1,j]) 
    elif i==N-1:
        return x[i,j]*(x[i,j-1] + x[0,j] + x[i,j+1] + x[i-1,j])
    elif j==N-1:
        return x[i,j]*(x[i,j-1] + x[i+1,j] + x[i,0] + x[i-1,j])
    else:
        return x[i,j]*(x[i,j-1] + x[i+1,j] + x[i,j+1] + x[i-1,j]) 

In [ ]:
#Gives us the energy of the state
def Energy(array):
    Enn=0
    for i in range(0,N):
        for j in range(0,N): 
            Enn = Enn - spin(i,j)
    return (1/2)*Enn
    

In [ ]:
#Gives us the magnetisation of the state
def mag(array):
    return array.sum()

<center> spin(i,j) gives us the associated energy of each spin; and Energy(x) gives us the hamiltonian of state and the mag(array) gives us the magnetisation of the state</center>

In [ ]:
#Gives us the average cluster size of +1 spins
def avg_size(arr):
    a = deepcopy(arr)
    a[a==-1]=0
    lw, num = measurements.label(a)
    area = measurements.sum(a, lw, index=arange(lw.max() + 1))
    return area[1:].mean()

In [ ]:
#This function implements the metropolis algorithm and calculates the needed quantities
def equilibrate(y,T):
    H = Energy(y)
    m = mag(y)
    steps=0
    E_calc=[H]
    M_calc=[m]
    sizes=[]
    for i in range(4*10**4):
        p = np.random.randint(0,N)
        q = np.random.randint(0,N)
        spin_i=y[p,q]
        dE = 2*spin(p,q)
        if dE <=0:
            y[p,q]=-y[p,q]
            E_new = H + dE
            m_new= m +(-2)*spin_i
            size=avg_size(y)
        else:
            temp=np.random.rand()
            prob = np.exp(-(dE)/T)
            if temp < prob:
                y[p,q]=-y[p,q]
                E_new = H + dE
                m_new= m +(-2)*spin_i
                size=avg_size(y)
            else:
                E_new = H
                m_new = m
                size=avg_size(y)
        H = E_new
        m = m_new
        E_calc.append(E_new)
        M_calc.append(m_new)
        sizes.append(size)
        
        
        
    
    E_calc = np.array(E_calc)
    M_calc = np.array(M_calc)
    
    
    return y, E_calc, sizes

In [ ]:
## The Following cells calculates the cluster sizes and then fits the log-log plot and then shows the plot.
sizess=[]
for i in range(20):
    x = np.random.rand(N,N)
    for i in range(0,N):
        for j in range(0,N):
            if x[i,j] < 0.6:
                x[i,j]=1
            else:
                x[i,j]=-1
    x,Es, sizes = equilibrate(x,0.6)
    sizess.append(sizes)

In [ ]:
npsz=np.array(sizess)
sizes=0
for i in range(len(npsz)):
    sizes += npsz[i]
sizes= np.array(sizes) / len(npsz)

In [ ]:
mc_steps=np.arange(0,4*10**4)

In [ ]:
def gro(x,a,b):
    return a+x*b

In [ ]:
y_data = np.array(np.log(sizes)[4*10**3:],dtype=float)
x_data = np.array(np.log(mc_steps)[4*10**3:],dtype = float)
popt, pcov = curve_fit(gro, x_data, y_data, maxfev=10**6)

In [ ]:
plt.plot((mc_steps), (sizes),label='Simulation Data')
plt.plot(x_data,popt[0]+x_data*popt[1],label='$y=3.52+0.31x$')
plt.xlabel('Monte Carlo Steps \n towards equilibrium $k_BT=0.6$')
plt.ylabel('Average Positive Spin Cluster Size')
plt.title('Log-Log Plot')
plt.legend()

In [ ]:
#Defining the two-point correlation function
def g(x,r):
    sum = 0
    for k in range(10**5):
        i = np.random.randint(0,N)
        j= np.random.randint(0,N)
        sum += x[i,j]*(x[i-r,j])
    return sum/(10**5)

In [ ]:
#The following parts gets the two-point-correlation values for different temperatures,
#fits it to get the correlation lengths at different temperatures then plots it
g_T_s=[]
r_s= [1,2,3,4,5,6,7,8,9,10]
T_s = np.arange(0.1,5,0.1)
for T in T_s:
    y=equilibrate(x,T)
    g_ss=[]
    for i in range(0,10):
        g_s=[]
        for r in r_s:
            g_r = g(y,r)
            g_s.append(g_r)
        g_ss.append(g_s)
    g_ss_np=np.array(g_ss)
    g_s_f=[]
    temp=0
    for i in range(len(g_ss)):
        temp+=g_ss_np[i]
    g_s_f=np.array(temp)/len(g_ss)
    g_T_s.append(g_s_f)

In [ ]:
def func(x,a,b,c,d):
    return c+ a*np.exp(-x/b)*x**(-d)

In [ ]:
x_data = np.array(r_s,dtype=float)
corr_lengths =[]
hm=[]
for i in range(len(g_T_s)):
    y_data = np.array(g_T_s[i],dtype = float)
    popt, pcov = curve_fit(func, x_data, y_data, maxfev=2000000)
    corr_lengths.append(popt[1])
    hm.append(popt)

In [ ]:
plt.plot(T_s,corr_lengths,'o--')
plt.xlabel('$k_BT$')
plt.ylabel('Correlation Length')

In [ ]:
def func2(x,b):
    return abs(x-2.25)**(-b)

In [ ]:
popt, pcov = curve_fit(func2,T_s[20:40],corr_lengths[20:40], maxfev=2*10**6)

In [ ]:
fit_corr=abs(T_s-2.25)**(-popt[0])

In [ ]:
plt.plot(T_s[20:40],corr_lengths[20:40],'o--',label='Computed Data')
plt.plot(T_s[20:40], fit_corr[25:40],label='Fitted plot, $f(T)=|T-2.25|^{-0.96}$')
plt.legend()
plt.xlabel('$K_BT$')
plt.ylabel('Correlation Length')

In [ ]:
#After equilibration we use this to get the values of the observables
def calculate(x,T):
    H = Energy(x)
    m = mag(x)
    steps=0
    E_calc=[H]
    M_calc=[m]
    
    for i in range(10**3):
        p = np.random.randint(0,N)
        q = np.random.randint(0,N)
        spin_i=x[p,q]
        steps = steps +1
        dE = 2*spin(p,q)
        if dE <=0:
            x[p,q]=-x[p,q]
            E_new = H + dE
            m_new= m +(-2)*spin_i
        else:
            temp=np.random.rand()
            prob = np.exp(-(dE)/T)
            if temp < prob:
                x[p,q]=-x[p,q]
                E_new = H + dE
                m_new= m +(-2)*spin_i
            else:
                E_new = H
                m_new= m
        H = E_new
        m = m_new
        E_calc.append(E_new)
        M_calc.append(m_new)

    E_calc = np.array(E_calc)
    M_calc = np.array(M_calc)
    
    E_avg= E_calc.mean()
    C= E_calc.std()/T**2
    M_avg= M_calc.mean()
    Chi = M_calc.std()/T
    
    M_avg = M_avg/ (N**2)
    
    return [E_avg , M_avg, C, Chi]


In [ ]:
#Defining the autocorrelation function
def corr(arr,dt):
    t_max = len(arr)
    return np.mean(arr[dt:t_max]*arr[:t_max-dt]) - np.mean(arr[dt:t_max])*np.mean(arr[:t_max-dt])

In [ ]:
y=np.array(x)
y, m_s = equilibrate(y,1.4)

In [ ]:
#The following parts calculates,plots the autocorrelation function at different temperatures.
times = np.arange(1,10**4,1)
corr_s = []
for i in times:
    cf = corr(m_s,i)
    corr_s.append(cf)
corr_s=np.array(corr_s) / corr_s[0]

In [ ]:
plt.plot(times,corr_s)
plt.xlabel('Lag time')
plt.ylabel('C(t)')

In [ ]:
c_T_s=[]
times = np.arange(1,5*10**3,1)
T_s = [1.5,2,2.25,3.5,4.25]
for T in T_s:
    y=equilibrate(x,T)
    e_avg,m_avg,c,chi, m_calc = calculate(x,T)
    corr_s = []
    for i in times:
        cf = corr(m_calc,i)
        corr_s.append(cf)
    c_T_s.append(corr_s)

In [ ]:
plt.plot(times,c_T_s[0],label='$k_BT=1.5$')
plt.plot(times,c_T_s[1],label='$k_BT=2$')
plt.plot(times,c_T_s[2],label='$k_BT=2.25$')
plt.plot(times,c_T_s[3],label='$k_BT=3.5$')
plt.plot(times,c_T_s[4],label='$k_BT=4.25$')
plt.xlabel('Lag time, $t$')
plt.ylabel('Correlation function, $C(t)$')
plt.legend()

In [ ]:
#The following fits the auto-correlation functions to get the correlation time at different temperatures, then plots it.
def func2(x,a,b):
    return a*np.exp(-x/b)

In [ ]:

T_s = np.arange(0.1,5,0.1)
aladata=[]
for T in T_s:
    x=equilibrate(x,T)
    e_avg,m_avg,c,chi, m_calc = calculate(x,T)
    corr_s = []
    for i in times:
        cf = corr(m_calc,i)
        corr_s.append(cf)
    corr_s = np.array(corr_s)
    aladata.append(corr_s)

In [ ]:
x_data = np.array(times, dtype = float)
lengths=[]
for i in range(0,49):
    y_data = np.array(aladata[i], dtype=float)
    popt, pcov = curve_fit(func2, x_data , y_data , maxfev=20000)
    lengths.append(popt[1])

In [ ]:
plt.plot(T_s,lengths,'o--')
plt.xlabel('$k_BT$')
plt.ylabel('Correlation Time')

In [ ]:
temp=np.arange(0.1,5,0.1) #Generating a set of temperatures 

In [ ]:
E_s=[]
M_s=[]
C_s=[]
Chi_s=[]
for T in temp:
    x=equilibrate(x,T)
    E_avg,M_avg, C, Chi=calculate(x,T)
    E_s.append(E_avg)
    M_s.append(M_avg)
    C_s.append(C)
    Chi_s.append(Chi)
#Running the metropolis algorithm on the state at different temperatures and plotting these

In [ ]:
plt.plot(temp,E_s, 'o--')
plt.xlabel('$k_bT$')
plt.ylabel('$E_{avg}$')

In [ ]:
plt.plot(temp, Chi_s,'o--')
plt.xlabel('$k_bT$')
plt.ylabel('$\chi$')